# Item 1: Know Which Version of Python You're Using

In [1]:
import sys
print(sys.version_info)
print(sys.version)

sys.version_info(major=3, minor=7, micro=3, releaselevel='final', serial=0)
3.7.3 (default, Mar 27 2019, 09:23:15) 
[Clang 10.0.1 (clang-1001.0.46.3)]


# Item 2: Follow the PEP 8 Style Guide

# Item 3: Know the Differences Between bytes, str, and unicode

* bytes - contains raw 8-bit values
* str - contains unicode characters
* Unicode
    * most common encoding is UTF-8
    * str instances don't have associated binary encoding
        * to convert Unicode to binary use str.encode()
        * to convert binary to Unicode use bytes.decode()
* When coding use Unicode character types (i.e. str)
    * code should accept alternative text encoding (e.g. Latin-1)
    * but output would always be UTF-8
    * MAKE SURE ENCODING IS CORRECT!

In [4]:
# File handles using open() defaults to UTF-8
# The following breaks

import os

with open('/tmp/random.bin', 'w') as f:
    f.write(os.urandom(10))

TypeError: write() argument must be str, not bytes

In [5]:
# Instead change to write binary mode ('wb') instead of write character mode ('w')

with open('/tmp/random.bin', 'wb') as f:
    f.write(os.urandom(10))
    
# same thing with 'rb' vs 'r'

# Item 4: Write Helper Functions Instead of Complex Expressions

* Instead of pithy unreadable oneliners move complex expressions into helper functions especially if same logic is used repeatedly
* if/else is more readable than using or or and in expressions

In [6]:
from urllib.parse import parse_qs
my_values = parse_qs('red=5&blue=0&green=', keep_blank_values=True)
print(repr(my_values))

{'red': ['5'], 'blue': ['0'], 'green': ['']}


In [7]:
print(str(my_values))
# repr is for developers (unambiguous, representation of python object. use eval to get it back). 
# str is for customers (readable, object in text form)

{'red': ['5'], 'blue': ['0'], 'green': ['']}


In [8]:
print('Red: ', my_values.get('red')) 
print('Green: ', my_values.get('green')) 
print('Opacity: ', my_values.get('opacity'))

Red:  ['5']
Green:  ['']
Opacity:  None


In [9]:
red = my_values.get('red', [''])[0] or 0
green = my_values.get('green', [''])[0] or 0 
opacity = my_values.get('opacity', [''])[0] or 0 
print(f"Red:     {red}")
print(f"Green:   {green}")
print(f"Opacity: {opacity}")

# get returns second argument if key doesn't exist in dictionary. in this case second argument is ['']
# [0] to get 0th element of [''] which is ''
# '' evaluates to False
# False or 0 yields 0

Red:     5
Green:   0
Opacity: 0


In [16]:
False or 0

0

In [18]:
# This is difficult to read. Also what if paramter isn't an integer?
# Need to convert to integer
red = int(my_values.get('red', [''])[0])

# This is harder to read

In [20]:
# Better to split into separate lines
red = my_values.get('red', [''])
red = int(red[0]) if red[0] else 0
print(red)

5


In [22]:
# ternary if else is not as clear as multiline if else
green = my_values.get('green', [''])
if green[0]:
    green = int(green[0])
else:
    green = 0
    
# much easier to read

In [24]:
# since same logic is used multiple times, it's better to write a helper function
def get_first_int(values, key, default=0):
    found = values.get(key, [''])
    if found[0]:
        found = int(found[0])
    else:
        found = default
        
    return found

green = get_first_int(my_values, 'green')
print(green)

0


# Item 5: Know How to Slice Sequences

* Avoid being verbose. Don't supply 0 for start index or length of sequence for end index. Just leave out number
* Slicing forgives start or end indexes that are out of bounds
* Assigning to a list slice will replace range in original sequence even if length is different

In [31]:
# Any class with __getitem__ and __setitem__ can be used to slice
# Slicing can be done on list, str, bytes
# Basic syntax is somelist[start:end]
# start is inclusive, end is exclusive

In [32]:
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print("First four: ", a[:4])
print("Last four: ", a[-4:])
print("Middle two: ", a[3:-3])

First four:  ['a', 'b', 'c', 'd']
Last four:  ['e', 'f', 'g', 'h']
Middle two:  ['d', 'e']


In [36]:
# When slicing from start of list leave out 0
assert a[:5] == a[0:5]

In [39]:
# When slicing to end of list leave out final index. It's redundant
assert a[5:] == a[5:len(a)]

In [40]:
# Using negative numbers helps doing offsets relative to end of list
print(a[:])
print(a[:5])
print(a[:-1])
print(a[4:])
print(a[-3:])
print(a[2:5])
print(a[2:-1])
print(a[-3:-1])

['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
['a', 'b', 'c', 'd', 'e']
['a', 'b', 'c', 'd', 'e', 'f', 'g']
['e', 'f', 'g', 'h']
['f', 'g', 'h']
['c', 'd', 'e']
['c', 'd', 'e', 'f', 'g']
['f', 'g']


In [ ]:
# Slicing allows start and end indexes that are not in bounds

In [27]:
lst = list(range(10))
print(lst)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [29]:
lst[:12]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [30]:
lst[-20:]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [41]:
# By contrast. Accessing same index causes an exception
lst[-20]

IndexError: list index out of range

In [42]:
# Slicing by negative index can be weird. somelist[-n:] works if n > 1, 
# but if n == 0, then somelist[-0:] returns copy of original list
print(lst[-3:])
print(lst[-0:])

[7, 8, 9]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [46]:
# Result of slicing is a whole new list
# References to objects from original list are maintained
# Meaning modifying result of slicing won't affect original list

b = a[4:]
print(f"Before change to b:      ", b)
b[1] = 99
print(f"After change to b:       ", b)
print(f"No change to original a: ", a)

Before change to b:       ['e', 'f', 'g', 'h']
After change to b:        ['e', 99, 'g', 'h']
No change to original a:  ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']


In [49]:
# In tuple assignments length of assignments are important
a, b = lst[:2]
print(lst)
print(a)
print(b)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
0
1


In [50]:
a, b = lst[:1]

ValueError: not enough values to unpack (expected 2, got 1)

In [56]:
# In slice assignments length doesn't need to be the same
# The values before and after slice is preserved
# The lists grows or shrinks to accommodate new values
lst = list(range(10))
print(f"Before {lst}")
print(f"Before length: {len(lst)}")
lst[2:7] = ['a', 'b', 'c'] # slice has 5 elements, but assignment only has 3
print(f"After {lst}") # list shrinks by two
print(f"After length: {len(lst)}")

Before [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Before length: 10
After [0, 1, 'a', 'b', 'c', 7, 8, 9]
After length: 8


In [58]:
# If you leave out start and end indices when slicing you get a copy of original list
a = list(range(10))
b = a[:]
assert b == a and b is not a

In [66]:
# If you assign a slice with no start or end indices
# Entire content is replaced with a copy of what's referenced
# A new list is not allocated

a = list(range(10))
b = a
print(f"Before {a}")
a[:] = [101, 102, 103]
assert a is b # Still the same object
print(f"After {a}") # But content is replaced

Before [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
After [101, 102, 103]


# Item 6: Avoid Using start, end, and stride in a Single Slice

* Specifying start, end, and stride in a slice is confusing
* Use positive stride values in slices without start or end indices. Avoid negative strides if possible
* Avoid using start, end, and stride in a single slice
* Instead consider using two assignments (one for slice, one for stride) or use islice from itertools

In [71]:
# Slicing also has a stride argument in form of somelist[start:end:stride]
# Stride gets every nth item, make it easy to get even or odd indices

a = ['red', 'orange', 'yellow', 'green', 'blue', 'purple']
odds = a[::2]
evens = a[1::2]
print(odds)
print(evens)

['red', 'yellow', 'blue']
['orange', 'green', 'purple']


In [72]:
# Strides have weird behavior
# This works for reversing a byte string

x = b'mongoose'
y = x[::-1]
print(y)

b'esoognom'


In [76]:
# But breaks for UTF-8
w = '안녕하세요'
x = w.encode('utf-8')
print(x)
y = x[::-1]
print(y)
z = y.decode('utf-8')

b'\xec\x95\x88\xeb\x85\x95\xed\x95\x98\xec\x84\xb8\xec\x9a\x94'
b'\x94\x9a\xec\xb8\x84\xec\x98\x95\xed\x95\x85\xeb\x88\x95\xec'


UnicodeDecodeError: 'utf-8' codec can't decode byte 0x94 in position 0: invalid start byte

In [77]:
# -1 stride is useful, but what bout -2?
a = ['a', 'b', 'c', 'd', 'e', 'f', 'g', 'h']
print(a[::2]) # select every second item starting from beginning
print(a[::-2]) # select every second item starting at end and moving backwards

['a', 'c', 'e', 'g']
['h', 'f', 'd', 'b']


In [78]:
# What about 2::2 or -2::-2 or -2:2:-2 or 2:2:-2
print(a[2::2])
print(a[-2::-2])
print(a[-2:2:-2])
print(a[2:2:-2])

['c', 'e', 'g']
['g', 'e', 'c', 'a']
['g', 'e']
[]


In [83]:
a[-2:2]

[]

In [80]:
a[-2:2:-2]

['g', 'e']

In [85]:
# Point is it's confusing
# Instead split into two -> slice and stride
b = a[::2]
print(b)
c = b[1:-1]
print(c)

['a', 'c', 'e', 'g']
['c', 'e']


# Item 7: Use List Comprehensions Instead of map and filter

* List comprehensions are clearer than map and filter built-in functions because no need for lambda
* List comprehensions allow you to skip items from input list. map() can do this, but you need filter()
* Dictionaries and sets also support comprehension

In [87]:
a = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [88]:
# Example list comprehsion
squares = [x**2 for x in a]
print(squares)

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [91]:
# Example using map
squares = map(lambda x: x**2, a)
print(squares)
print(list(squares))

[1, 4, 9, 16, 25, 36, 49, 64, 81, 100]


In [92]:
# Unlike map, list comprehensions allow you to filter items from input list using conditionals
even_squares = [x**2 for x in a if x%2 == 0]
print(even_squares)

[4, 16, 36, 64, 100]


In [95]:
print(1%2)
print(2%2)
print(3%2)
print(4%2)

1
0
1
0


In [96]:
# Filter can be combined with map to get the same result
alt = map(lambda x: x**2, filter(lambda x: x%2 == 0, a))
assert even_squares == list(alt)

In [97]:
# Dictionaries can also use comprehensions
chile_ranks = {'ghost': 1, 'habanero': 2, 'cayenne': 3}
rank_dict = {rank:name for name, rank in chile_ranks.items()}
chile_len_set = {len(name) for name in rank_dict.values()}
print(rank_dict)
print(chile_len_set)

{1: 'ghost', 2: 'habanero', 3: 'cayenne'}
{8, 5, 7}


# Item 8: Avoid More Than Two Expressions in List Comprehensions

* List comprehensions support multiple levels of loops and multiple conditions per loop level
* But list comprehensions with more than two expressions is difficult to read and should be avoided

In [99]:
matrix = [[1,2,3], [4,5,6], [7,8,9]]
flat = [x for row in matrix for x in row] # notice outer loop comes first
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [101]:
# This is equivalent to
lst = []
for row in matrix:
    for x in row:
        lst.append(x)
print(lst)

[1, 2, 3, 4, 5, 6, 7, 8, 9]


In [102]:
# Another use for multiple loops is replicating two-level deep layout of input list
squared = [[x**2 for x in row] for row in matrix]
print(squared)

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]


In [104]:
# This is equivalent to
lst = []
for row in matrix:
    inner_lst = []
    for x in row:
        inner_lst.append(x**2)
    lst.append(inner_lst)
print(lst)

[[1, 4, 9], [16, 25, 36], [49, 64, 81]]


In [105]:
# If you add another layer, it gets confusing quickly
my_lists = [
    [[1,2,3], [4,5,6]],
    [[7,8,9], [10,11,12]],
    [[13,14,15], [16,17,18]]
]

In [108]:
flat = [x for sublist1 in my_lists for sublist2 in sublist1 for x in sublist2]
print(flat)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [110]:
# This is equivalent to
lst = []
for sublist1 in my_lists:
    for sublist2 in sublist1:
        for x in sublist2:
            lst.append(x)
print(lst)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]


In [112]:
# Append vs extend
# Append adds the whole element to the end
x = [1, 2, 3]
x.append([4, 5])
print (x)

[1, 2, 3, [4, 5]]


In [113]:
# Extend adds elements from the iterable to end
x = [1, 2, 3]
x.extend([4, 5])
print (x)

[1, 2, 3, 4, 5]


In [120]:
# List comprehensions also support multiple if conditions
# Multiple conditions at the same level are like an "and"
# e.g. let's say you want to filter al ist of numbers to only even values and numbers > 4
a = list(range(1,11))
print(a)
b = [x for x in a if x > 4 if x%2 == 0]
print(b)
c = [x for x in a if x > 4 and x%2 == 0]
print(c)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]
[6, 8, 10]
[6, 8, 10]


In [121]:
# Conditions can be specified at each level of looping after the for expression
# e.g. if you want to filter matrix so only cells remaining are divisible by 3 in rows that sum to 10 or higher

matrix = [[1,2,3],[4,5,6],[7,8,9]]
filtered = [[x for x in row if x%3 == 0] for row in matrix if sum(row)>=10]
print(filtered)

[[6], [9]]


In [124]:
# This is equal to
outer_lst = []
for row in matrix:
    inner_lst = []
    if sum(row) >= 10:
        for x in row:
            if x%3 == 0:
                inner_lst.append(x)
        outer_lst.append(inner_lst)
print(outer_lst)

[[6], [9]]


In [125]:
# Rule of thumb is avoid using more than two expressions in a list comprehension
# e.g. two conditions, two loops, or one condition and one loop
# If there are more than two conditions use if and for statements and write a helper function

# Item 9: Consider Generator Expressions for Large Comprehensions

* List comprehensions can cause problems if inputs are large enough to use too much memory
* Generator expressions avoid memory issues because they produce outputs one at a time as an iterator
* Generator expressioned can be made by passing iterator from one generator expression into the for expression of another
* Generator expressions execute very quickly when chained together

In [152]:
s = """hello
this is an
example text file
that is going to be used 
in the
next exercise
"""

with open('/tmp/my_file.txt', 'w') as f:
    f.write(s)

In [153]:
values = [len(x) for x in open('/tmp/my_file.txt')]
print(values)

[6, 11, 18, 26, 7, 14]


In [154]:
# to handle large files that can't be held in memory, Python provides generator expressions
# Generator expressions are a generalization of list comprehensions and generators
# Generators don't materialize whole output sequence when run
# Instead they evalute an iterator that yields one item at a time from the expression
# Make a generator by using list comprehension syntax with ()

# this is the same as above
it = (len(x) for x in open('/tmp/my_file.txt'))
print(it)

<generator object <genexpr> at 0x106288ed0>


In [155]:
# advance one element at a time
print(next(it))
print(next(it))

6
11


In [156]:
# generators can be composed together
roots = ((x, x**0.5) for x in it)
print(roots)

<generator object <genexpr> at 0x106288f48>


In [157]:
print(next(roots))
print(next(roots))

(18, 4.242640687119285)
(26, 5.0990195135927845)


In [158]:
it = (len(x) for x in open('/tmp/my_file.txt'))
roots = ((x, x**0.5) for x in it)
print(next(roots))
print(next(roots))

(6, 2.449489742783178)
(11, 3.3166247903554)


In [159]:
# This is equivalent to
roots_lst = [x**0.5 for x in values]
print(roots_lst)

[2.449489742783178, 3.3166247903554, 4.242640687119285, 5.0990195135927845, 2.6457513110645907, 3.7416573867739413]


In [162]:
# When using chain functionality for large inputs use generators
# But remember iterators returned by generator expressions are stateful (i.e. they hold state), 
# so don't use them more than once

# Item 10: Prefer enumerate Over range

* enumerate is a concise syntax for looping over iterators and getting index of each item as you go
* enumerate is better than looping over a range and indexing into sequence
* supply second parameter to enumerate to specify start number (0 is default)

In [169]:
from random import randint

random_bits = 0
for i in range(64):
    if randint(0,1):
        random_bits |= 1 << i # I have no idea why is here

In [182]:
i

63

In [166]:
random_bits

1574779709798626052

In [181]:
random_bits = 0
random_bits |= 1
random_bits

1

In [179]:
# |= is inplace OR
s1 = {1, 2, 3}
s2 = {4, 5, 6}
s1 | s2
s1

{1, 2, 3}

In [180]:
s1 = {1, 2, 3}
s2 = {4, 5, 6}
s1 |= s2
s1

{1, 2, 3, 4, 5, 6}

In [183]:
# If you have an iterator directly loop over the sequence
flavor_list = ['vanilla', 'chocolate', 'pecan', 'strawberry']
for flavor in flavor_list:
    print(f"{flavor} is delicious")

vanilla is delicious
chocolate is delicious
pecan is delicious
strawberry is delicious


In [184]:
# Sometimes you want to use the index and value
# One way is to use the range
for i in range(len(flavor_list)):
    flavor = flavor_list[i]
    print(f"{i+1}: {flavor}")

1: vanilla
2: chocolate
3: pecan
4: strawberry


In [185]:
# It looks cleaner to use enumerate
for i, flavor in enumerate(flavor_list):
    print(f"{i+1}: {flavor}")

1: vanilla
2: chocolate
3: pecan
4: strawberry


In [187]:
# You can get rid of i+1 by specifying what number enumerate should begin counting
for i, flavor in enumerate(flavor_list, 1):
    print(f"{i}: {flavor}")

1: vanilla
2: chocolate
3: pecan
4: strawberry


In [163]:
# use df.iterrows() to iterate in pandas
# for index, row in df.iterrows():
#     ...

# Item 11: Use zip to Process Iterators in Parallel

* zip can be used to iterate over multiple iterators in parallel
* zip is a lazy generator that produces tuples
* zip truncates output silently if iterators have different lengths
* zip_longest() from itertools lets you iterate over multiple iterators in parallel regardless of lengths

In [2]:
# sometimes you want to iterate over related iterators
names = ['Cecilia', 'Lise', 'Marie']
letters = [len(n) for n in names]
print(letters)

[7, 4, 5]


In [3]:
zip(names, letters)

In [4]:
list(zip(names, letters))

[('Cecilia', 7), ('Lise', 4), ('Marie', 5)]

In [192]:
# One way is to iterate over length of names
longest_name = None
max_letters = 0

for i in range(len(names)):
    count = letters[i]
    if count > max_letters:
        longest_name = names[i]
        max_letters = count

print(longest_name)

# But this is so verbose

Cecilia


In [195]:
# A cleaner way is to use enumerate for the indexes
longest_name = None
max_letters = 0
for i, name in enumerate(names):
    count = letters[i]
    if count > max_letters:
        longest_name = name
        max_letters = count
print(longest_name)

Cecilia


In [199]:
# an even cleaner method is zip
longest_name = None
max_letters = 0
for name, count in zip(names, letters):
    if count > max_letters:
        longest_name = name
        max_letters = count
print(longest_name)

Cecilia


In [200]:
# if input iterators are of different lengths, you silently get weird results
# e.g. Rosalind isn't in names
names.append('Rosalind') # names now has four elements ['Cecilia', 'Lise', 'Marie', 'Rosalind']
for name, count in zip(names, letters):
    print(name)

Cecilia
Lise
Marie


In [203]:
# What happened to Rosalind?
# zip yields tuples until a wrapped iterator is exhausted
# use zip_longest from itertools if you're not sure the input iterators are of equal length
from itertools import zip_longest
for name, count in zip_longest(names, letters):
    print(f"name: {name}, count: {count}")

name: Cecilia, count: 7
name: Lise, count: 4
name: Marie, count: 5
name: Rosalind, count: None


# Item 12: Avoid else Blocks After for and while Loops

* Python has special syntax that allows else blocks to immediately follow for and while loop interior blocks
* else block only runs if loop body didn't encounter a break
* avoid using else blocks after loops because behavior is weird

In [204]:
# Else block runs after loop finishes
for i in range(3):
    print(f"Loop: {i}")
else:
    print('Else block!')

Loop: 0
Loop: 1
Loop: 2
Else block!


In [206]:
# But a break will skip the else block
for i in range(3):
    print(f"Loop: {i}")
    if i == 1:
        break
else:
    print('Else block!')
# else is not a finally!

Loop: 0
Loop: 1


In [207]:
# else blocks run immediately if you loop over an empty sequence
for x in []:
    print("Never runs!")
else:
    print("For else block!")

For else block!


In [208]:
# Else blocks also run when while loops are initially False
while False:
    print("Never runs")
else:
    print('While else block')

While else block


In [211]:
# Else blocks are useful when you use loops to search for something
# are these two numbers comprime?
a = 4
b = 9
for i in range(2, min(a,b) + 1):
    print(f"Testing {i}")
    if a % i == 0 and b % i == 0:
        print('Not coprime')
        break
else:
    print('Yes they are Comprime')

Testing 2
Testing 3
Testing 4
Yes they are Comprime


In [210]:
min(a,b)

4

In [212]:
# But it's better to write a helper function
def coprime(a,b):
    for i in range(2, min(a,b) + 1):
        if a % i == 0 and b % i == 0:
            return False
    return True

In [213]:
coprime(4,9)

True

In [215]:
# Another way is to have a result variable that indicates whether you've found what you're looking for in a loop
def coprime2(a,b):
    is_coprime = True
    for i in range(2, min(a,b)+1):
        if a % i == 0 and b % i == 0:
            is_coprime = False
            break
    return is_coprime
coprime2(4,9)

True

In [216]:
# Basically avoid else block after loops

# Item 13: Take Advantage of Each Block in try/except/else/finally


* try/finally let's you run cleanup code regardless of exceptions raised in try block
* else block helps you minimize amount of code in try blocks and distinguishes success case from try/except blocks
* else blocks can also be used for additional actions after a successful try block, but before final cleanup in finally

## Finally Blocks

In [220]:
s = """hello
this is an
example text file
that is going to be used 
in the
next exercise
"""

with open('/tmp/random_data.txt', 'w') as f:
    f.write(s)

In [222]:
handle = open('/tmp/random_data.txt')  # May raise IOError
try:
    data = handle.read() # May raise UnicodeDecodeError
finally:
    handle.close() #Always runs after try
data

'hello\nthis is an\nexample text file\nthat is going to be used \nin the\nnext exercise\n'

In [223]:
handle = open('/tmp/random_data2.txt')
try:
    data = handle.read()
finally:
    handle.close()
data

FileNotFoundError: [Errno 2] No such file or directory: '/tmp/random_data2.txt'

## Else Blocks

In [225]:
# Use try/except/else to make it clear which exceptions are handled by code, and which propagate up
# If try block doesn't raise an exception else runs
# Else blocks help minimize code in try block and improves readability

def load_json_key(data, key):
    try:
        result_dict = json.loads(data) # May raise ValueError
    except ValueError as e:
        raise KeyError from e
    else:
        return result_dict[key] # May raise KeyError

## Everything Together

In [ ]:
UNDEFINED = object()

def divide_json(path):
    handle = open(path, 'r+')  # May raise IOError
    try:
        data = handle.read()   # May raise UnicodeDecodeError
        op = json.loads(data)  # May raise ValueError
        value = (
            op['numerator'] /  # May raise ZeroDivisionError
            op['denominator']
        )
    except ZeroDevisionError as e:
        return UNDEFINED
    else:
        op['result'] = value
        result = json.dumps(op)
        handle.seek(0)         # seek starts at character 0
        handle.write(result)   # May raise IOError
        return value
    finally:
        handle.close()         # Always runs